In [6]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Neural")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Neural, master=local[*]) created by __init__ at <ipython-input-1-3ea5fb6248f2>:7 

In [7]:
# Package imports
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model
from sklearn.model_selection import train_test_split

In [8]:
np.random.seed(0)
X, y = sklearn.datasets.make_moons(200, noise=0.20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.43, random_state=42)

In [10]:
input_layer=2
output_layer=10
nn_input_dim = 2 # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality

#gradient decent
eta=0.01
reg_lambda=0.01

In [15]:
# This function learns parameters for the neural network and returns the model.
# - nn_hdim: Number of nodes in the hidden layer
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations
def build_model(X,y,nn_hdim, num_passes, print_loss):
    # This is what we return at the end
    model = {}
    W1,b1,W2,b2=initialize_weights(nn_input_dim, nn_hdim,nn_output_dim)
    # Gradient descent. For each batch...
    for i in range(0,num_passes):
        #Forward propagation   
        k=0
        for x in X:
            
            #z1,a1,z2,probs=forward(W1,b1,W2,b2,x)
        
            dW1,db1,dW2,db2=feedforward(W1,b1,W2,b2,x,y[k])
#             # Backpropagation
#             delta3 = probs
#             #print(len(x))
#             delta3[range(len(x)-1), y[k]] -= 1
#             dW2 = (a1.T).dot(delta3)
#             db2 = np.sum(delta3, axis=0, keepdims=True)
#             delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
#             m=np.reshape(x,(2,1))
#             dW1 = np.dot(m, delta2)
#             db1 = np.sum(delta2, axis=0)
            k=k+1
            # Add regularization terms (b1 and b2 don't have regularization terms)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1

            # Gradient descent parameter update
        W1 += -eta * dW1
        b1 += -eta * db1
        W2 += -eta * dW2
        b2 += -eta * db2
            
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
            
            print("Loss after iteration %i: %f" %(i, calculate_loss(model,X,y)))  
            
    return model


In [ ]:
# Build a model with a 3-dimensional hidden layer
num_examples = len(X_train)
model = build_model(X_train,y_train,40,100, False)

In [ ]:
y=np.array(([[1],[2]]))
y_pred=[]
for i in X_train:
    y_pred.append(predict(model,i))

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.85087719298245612

In [379]:
y_pred.append(predict(model,np.array([1,2])))

(1, 2)
(2, 40)


In [76]:
def convert(x):
    y=0
    for i in x:
        x[y] = float(i)
        y+=1
    return x

In [200]:
trainRDD = sc.textFile("C:/Users/saika/Desktop/files/TestFiles/neural.csv").filter(lambda x:x[0]!=",").map(lambda x:x.split(",")[1:])
trainRDD = trainRDD.map(convert)

In [202]:
trainRDD.count()

50

In [35]:
def calculate_loss(model,X,y):
    W1, W2 = model['W1'],  model['W2']
    z1,a1,z2,probs=forward(W1,W2,X)
    # Calculating the loss
    corect_logprobs = -np.log(probs[range(num_examples), y])
    data_loss = np.sum(corect_logprobs)
    # Add regulatization term to loss (optional)
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss

In [34]:
def initialize_weights(dim_input, dim_hidden,dim_output):
    # Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    w1 = np.random.randn(dim_input, dim_hidden) / np.sqrt(dim_input)
    w0 = np.random.randn(dim_hidden, dim_output) / np.sqrt(dim_hidden)
    return w1,w0

In [33]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [32]:
def softmax(z):
    exp_scores = np.exp(z)
    return (exp_scores / np.sum(exp_scores, axis=1, keepdims=True))

In [31]:
def forward(W1,W2,X):
    a1 = np.column_stack(( X))
    print(np.shape(a1))
    # hidden activations
    z1 = a1.dot(W1)
    a1 = np.tanh(z1)
    #a1 = np.vstack(( sigmoid(z1)))
    # output activations
    z2 = a1.dot(W2)
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    
    return z1,a1,z2,probs

In [190]:
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, W2= model['w1'], model['w2']
    # Forward propagation
    z1,a1,z2,probs=forward(W1,W2,x)
    return np.argmax(probs,axis=1)
    

In [206]:
def feedforward(W1,W2,x,y):
    a1 = np.column_stack((x))
    print(np.shape(a1))
    # hidden activations
    z1 = a1.dot(W1)
    a1 = np.tanh(z1)
    #a1 = np.vstack(( sigmoid(z1)))

    # output activations
    z2 = a1.dot(W2)
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    #back Propagation
    delta3 = probs
            #print(len(x))
#     delta3 = delta3.transpose()
    delta3[0][y] -= 1
    dW2 = (a1.T).dot(delta3)
    delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
    m=np.reshape(np.array(x),(3012,1))
    dW1 = np.dot(m, delta2)
    return np.array([dW1,dW2])

In [204]:
def Network(w1,w2, train_set, max_iter):
    Lambda = 10.01 # Regularization parameter (to avoid overfit issue)
    m = train_set.count()
    
    for ik in range(max_iter):
        eval_res = train_set.map(lambda x: feedforward(w1,w2,x[0:-1],x[-1]))
        
        # calculate derivation of weights via average bp results
        average_eval = eval_res.reduce(add) / train_set.count()
        dw1 = average_eval[0]
        dw2 = average_eval[1]
        
        
        # for all the weights, you should not apply regulization on the first column since they are for bias
        for i in range(w1.shape[0]):
            w1[i, 0] = w1[i, 0] - dw1[i, 0]
        for i in range(w1.shape[0]):
            for j in range(1, w1.shape[1]):
                # here learn rate is 1.0, Lambda is the Regularization parameter
                w1[i, j] = w1[i, j] - (dw1[i, j] + (Lambda/m) * w1[i, j])
        
        for i in range(w2.shape[0]):
            w2[i, 0] = w2[i, 0] - dw2[i, 0]
        for i in range(w2.shape[0]):
            for j in range(1, w2.shape[1]):
                w2[i, j] = w2[i, j] - (dw2[i, j] + (Lambda/m) * w2[i, j])
        
    model = { 'w1': w1, 'w2': w2}
    return model

In [209]:
from operator import add
nn_input_dim = 3012# input layer dimensionality
nn_output_dim = 10 # output layer dimensionality
nn_hdim=30
w1,w2=initialize_weights(nn_input_dim, nn_hdim,nn_output_dim)
#gradient decent
eta=0.01
reg_lambda=0.01
model=Network(w1,w2,trainRDD,20)

In [210]:
y_pred=trainRDD.map(lambda x:predict(model,x[0:-1])).collect()
accuracy_score(trainRDD.map(lambda x:x[-1]).collect(), y_pred)

0.12